In [1]:
# @title Default title text
!pip install langgraph langsmith langchain langchain_community
!pip install faiss-cpu
!pip install sentence-transformers
!pip install -U langchain-huggingface
!pip install -U langchain-google-genai
!pip install -U langchain_openai
!pip install -U openai
!pip install -U pinecone
!pip install flashrank

## PINECONE DB RETRIEVER

In [2]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from typing import List, Any
from pydantic import BaseModel, Field

from langchain.schema import BaseRetriever, Document
from langchain.load import dumps, loads
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


class PineconeDBRetriever(BaseRetriever, BaseModel):
    """
    A custom LangChain retriever for Pinecone.
    """
    index_name: str
    pinecone_api_key: str
    namespace: str
    top_k: int = 5
    index: Any = Field(None, exclude=True)

    def __init__(self, **data):
        """
        Initializes the Pinecone client and index.
        """
        super().__init__(**data)
        pc = Pinecone(api_key=self.pinecone_api_key)
        self.index = pc.Index(self.index_name)

    def _get_relevant_documents(self, query: str) -> List[Document]:
        """
        The core method to retrieve documents. LangChain's retriever system
        calls this method.

        Args:
            query (str): The user's question.

        Returns:
            List[Document]: A list of relevant documents from Pinecone.
        """
        # Pinecone's hosted embedding model will automatically embed the query text.
        results = self.index.search(
            namespace=self.namespace,
            query={
                "inputs": {"text": query},
                "top_k": self.top_k
            }
        )

        # Convert Pinecone's search results into LangChain Document objects.
        # TODO: Add additional fields as necessary.
        documents = []
        if results and 'result' in results and 'hits' in results['result']:
            for match in results['result']['hits']:
                # The actual text content is in the 'fields' dictionary
                page_content = match.get('fields', {}).get('text', '')
                # metadata = {"id": match.get("_id"), "score": match.get("_score")}
                metadata = {"id": match.get("_id")} # Removing score to allow easy serialization and help de-duplication

                doc = Document(
                    page_content=page_content,
                    metadata=metadata
                )
                documents.append(doc)

        return documents

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        """
        Asynchronous version of the document retrieval method.
        """
        # For simplicity, we'll just call the synchronous version.
        # For a production environment, you might want to use an async Pinecone client.
        return self._get_relevant_documents(query)

## RAG ORCHESTRATOR

In [3]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.runnables import RunnableConfig
from langchain_openai import ChatOpenAI
from pprint import pprint
from typing import Any, List
from langchain.load import dumps, loads

class RAGOrchestrator:
    """
    Orchestrates the RAG pipeline based on a given configuration.
    """
    def __init__(self, config: dict):
        """
        Initializes the orchestrator with a configuration dictionary.

        Args:
            config (dict): A dictionary containing settings for the RAG pipeline,
                           such as model name, index name, and retrieval strategy.
        """
        self.config = config
        self.debug = config.get("debug", False)
        self.llm = ChatOpenAI(
            model=config.get("llm_model", "gpt-4o-mini"),
            api_key=OPENAI_API_KEY
        )
        if config.get("retrieval_strategy") != "llm_only":
            self.retriever = PineconeDBRetriever(
                index_name=config.get("index_name"),
                pinecone_api_key=PINECONE_API_KEY,
                namespace=config.get("namespace"),
                top_k=config.get("top_k", 5)
            )

    # --- Debugging Helper ---
    def _print_debug(self, header: str, data: Any):
        if self.debug:
            print("\n" + "="*20)
            print(f"DEBUG: {header}")
            print("="*20)
            pprint(data)
        return data # Pass data through unchanged

    def _tap_and_log(self, x: dict) -> dict:
        """
        A helper method to print debug info and pass the input dictionary through unchanged.
        """
        self._print_debug("Final Context for LLM", x.get("context_str", "Context not available"))
        return x

    def _get_unique_union(self, documents: list[list]) -> List[Document]:
        """
        Takes a list of document lists, merges them, and removes duplicates.
        """
        # Serialize each document to a string to make them hashable
        flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
        # Use a set to get unique serialized documents
        unique_docs = list(set(flattened_docs))
        # Deserialize unique documents back into Document objects
        return [loads(doc) for doc in unique_docs]

    # --- Retrieval Strategy Helpers ---

    def _get_multi_query_chain(self):
        # Builds a chain that generates multiple queries and retrieves documents for each.
        # 1. Prompt for generating multiple queries
        template = """You are an AI language model assistant. Your task is to generate five
        different versions of the given user question to retrieve relevant documents from a vector
        database. By generating multiple perspectives on the user question, your goal is to help
        the user overcome some of the limitations of the cosine-based similarity search.
        Provide these alternative questions separated by newlines. Original question: {question}"""
        prompt_perspectives = ChatPromptTemplate.from_template(template)

        # 3. The chain for generating and retrieving
        generate_queries = (
            prompt_perspectives
            | self.llm
            | StrOutputParser()
            | (lambda x: x.split("\n"))
            | RunnableLambda(lambda x: [q for q in x if q.strip()])
            | RunnableLambda(lambda x: self._print_debug("Generated Queries", x))
        )

        retrieval_chain = generate_queries | self.retriever.map() | self._get_unique_union | RunnableLambda(lambda docs: self._print_debug("Retrieved Documents", docs))
        return retrieval_chain

    def _get_rag_fusion_chain(self):
        """Builds a chain for RAG Fusion with reciprocal rank fusion."""
        # 1. The multi-query generation is the same as above
        template = """You are an AI language model assistant. Your task is to generate five
        different versions of the given user question to retrieve relevant documents from a vector
        database. By generating multiple perspectives on the user question, your goal is to help
        the user overcome some of the limitations of the cosine-based similarity search.
        Provide these alternative questions separated by newlines. Original question: {question}"""
        prompt_perspectives = ChatPromptTemplate.from_template(template)

        generate_queries = (
            prompt_perspectives
            | self.llm
            | StrOutputParser()
            | (lambda x: x.split("\n"))
            | RunnableLambda(lambda x: [q for q in x if q.strip()])
            | RunnableLambda(lambda x: self._print_debug("Generated Queries", x))
        )

        # 2. Reranking with Reciprocal Rank Fusion
        def reciprocal_rank_fusion(results: list[list], k=60):
            fused_scores = {}
            for docs in results:
                for rank, doc in enumerate(docs):
                    doc_str = dumps(doc)
                    if doc_str not in fused_scores:
                        fused_scores[doc_str] = 0
                    fused_scores[doc_str] += 1 / (rank + k)

            # .item() converts [doc_str: score] pairs to a list of tuples [doc_str, score]
            # Sort by score in descending order (reverse=True)
            reranked_results = [
                (loads(doc), score)
                for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
            ]
            self._print_debug("Reranked Documents (RAG Fusion)", reranked_results)
            # Return only the documents, not the scores
            return [doc for doc, score in reranked_results]

        # 3. The RAG Fusion chain
        retrieval_chain = generate_queries | self.retriever.map() | reciprocal_rank_fusion
        return retrieval_chain

    def _get_decomposition_chain(self):
        # Builds a chain that decomposes a question into sub-questions.
        # 1. Prompt for generating sub-questions
        decomposition_template = """You are a helpful assistant that generates multiple sub-questions
        related to an input question. The goal is to break down the input into a set of sub-problems
        that can be answered in isolation. Generate multiple search queries related to: {question}
        Output (separated by newlines):"""
        prompt_decomposition = ChatPromptTemplate.from_template(decomposition_template)

        # 2. Chain to generate and clean up sub-questions
        generate_queries_decomposition = (
            prompt_decomposition
            | self.llm
            | StrOutputParser()
            | (lambda x: x.split("\n"))
            | RunnableLambda(lambda x: [q for q in x if q.strip()])
            | RunnableLambda(lambda x: self._print_debug("Decomposed Sub-questions", x))
        )

        # 3. The full retrieval chain using the decomposed questions
        retrieval_chain = (
            generate_queries_decomposition
            | self.retriever.map()
            | self._get_unique_union
            | RunnableLambda(lambda docs: self._print_debug("Retrieved Documents (Decomposition)", docs)
            ))
        return retrieval_chain

    def _get_step_back_chain(self):
        # Builds a chain that generates a general, "stepped-back" question and retrieves documents for it.
        # 1. Prompt to generate a more general, "stepped-back" question
        step_back_template = """You are an expert at world knowledge. Your task is to step back and
        paraphrase a question to a more generic step-back question, which is easier to answer.

        Here are a few examples:
        Original Question: What is the C29x CPU architecture in the F29H85x microcontroller?
        Step-Back Question: What are the technical specifications of the C29x CPU architecture?

        Original Question: Which TI device was recommended for automotive radar in the 2023 safety seminar?
        Step-Back Question: What are some common TI devices used for automotive radar applications?

        Original Question: {question}
        Step-Back Question:"""
        prompt_step_back = ChatPromptTemplate.from_template(step_back_template)

        # 2. Chain to generate the new question
        generate_step_back_query = (
            prompt_step_back
            | self.llm
            | StrOutputParser()
            #| (lambda x: x.split("\n"))
            #| RunnableLambda(lambda x: [q for q in x if q.strip()])
            | RunnableLambda(lambda x: self._print_debug("Generated Step-Back Question", x))
        )

        # 3. The full retrieval chain using the new question
        # This takes the original question, generates a new one, and retrieves docs with it
        retrieval_chain = generate_step_back_query | self.retriever | RunnableLambda(lambda docs: self._print_debug("Retrieved Documents (Step back)", docs))
        return retrieval_chain

    def _get_hyde_chain(self):
        # Builds a chain that generates a hypothetical document and uses it for retrieval.

        # 1. Prompt to generate a hypothetical document (a plausible answer)
        hyde_template = """Please write a passage to answer the user's question.
        This passage should be detailed and informative, as if it came from a technical document.
        The purpose is to create a rich text for a vector search.

        Question: {question}
        Passage:"""
        prompt_hyde = ChatPromptTemplate.from_template(hyde_template)

        # 2. Chain to generate the hypothetical document
        generate_hyde_document = (
            prompt_hyde
            | self.llm
            | StrOutputParser()
            | RunnableLambda(lambda x: self._print_debug("Generated Hypothetical Document", x))
        )

        # 3. The full retrieval chain: generate a hypothetical doc, then retrieve with it
        retrieval_chain = generate_hyde_document | self.retriever | RunnableLambda(lambda docs: self._print_debug("Retrieved Documents (HyDE)", docs))
        return retrieval_chain

    def invoke(self, question: str) -> dict:
        """
        Builds and invokes the RAG chain based on the configuration.

        Args:
            question (str): The user's question.

        Returns:
            dict: A dictionary containing the question, retrieved context, and the final answer.
        """
        strategy = self.config.get("retrieval_strategy", "simple")

        # LLM_only strategy does not use retrieval
        if strategy == "llm_only":
            self._print_debug("Strategy", "LLM Only (No RAG)")
            answer = self.llm.invoke(question).content
            return {
                "question": question,
                "answer": answer,
                "strategy": strategy,
                "context": "N/A" # No context was used
            }

         # --- For RAG-based strategies ---
        if strategy == "multi_query":
            retrieval_chain = self._get_multi_query_chain()
        elif strategy == "rag_fusion":
            retrieval_chain = self._get_rag_fusion_chain()
        elif strategy == "decomposition":
            retrieval_chain = self._get_decomposition_chain()
        elif strategy == "step_back":
            retrieval_chain = self._get_step_back_chain()
        elif strategy == "hyde":
            retrieval_chain = self._get_hyde_chain()
        # -----------------------------
        else: # Default to simple retrieval
            retrieval_chain = self.retriever | RunnableLambda(
                lambda docs: self._print_debug("Retrieved Documents (Simple)", docs)
            )

        final_prompt_template = """Answer the following question based only on the provided context...
        <context>{context}</context>
        Question: {question}"""
        final_prompt = ChatPromptTemplate.from_template(final_prompt_template)

        def format_docs(docs: List[Document]) -> str:
            return "\n\n".join(doc.page_content for doc in docs)

        # Define the main RAG processing steps
        rag_steps = (
            RunnablePassthrough.assign(
                context_str=itemgetter("context") | RunnableLambda(format_docs)
            )
            #| RunnableLambda(self._tap_and_log)
        )

        # Define the final chain that uses the processed steps
        rag_chain = (
            {"context": retrieval_chain, "question": RunnablePassthrough()}
            | rag_steps
            | {
                  "answer": (
                      lambda x: {"context": x["context_str"], "question": x["question"]}
                  ) | final_prompt | self.llm | StrOutputParser(),
                  "context": itemgetter("context"),
              }
        )

        result = rag_chain.invoke(question)

        return {
            "question": question,
            "answer": result['answer'],
            "strategy": strategy,
            "context": result['context']
        }

In [4]:
# --- Example Usage ---
# main.py
from pprint import pprint


# Get API keys using using dotenv
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY_SHIVAM")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Get API keys using google.colab
# from google.colab import userdata
# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# PINECONE_API_KEY = userdata.get('PINECONE_API_KEY_SHIVAM')


# --- Example 1: RAG Fusion with Debugging Enabled ---
print("--- Running with RAG Fusion Strategy (Debug Mode) ---")
fusion_config = {
    "llm_model": "gpt-4o-mini",
    "retrieval_strategy": "hyde",
    "index_name": "swru526-pine",
    "namespace": "example-namespace",
    "top_k": 5,
    "debug": True  # <-- Enable debugging
}

orchestrator_fusion = RAGOrchestrator(fusion_config)
question = "Which mmwave IC is the document about?"
result_fusion = orchestrator_fusion.invoke(question)
print("\n--- FINAL OUTPUT ---")
pprint(result_fusion)


# --- Example 2: LLM Only (No RAG) via Config ---
print("\n\n" + "--- Running with LLM Only Strategy ---")
llm_only_config = {
    "llm_model": "gpt-4o-mini",
    "retrieval_strategy": "llm_only",
    "debug": False # Debug flag works here too
}

orchestrator_llm_only = RAGOrchestrator(llm_only_config)
result_llm_only = orchestrator_llm_only.invoke(question)
print("\n--- FINAL OUTPUT ---")
pprint(result_llm_only)

--- Running with RAG Fusion Strategy (Debug Mode) ---

DEBUG: Generated Hypothetical Document
('The document focuses on the latest advancements and specifications of the '
 'AWR2843 mmWave Integrated Circuit (IC), a highly versatile and efficient '
 'device designed for a wide range of applications in automotive radar, '
 'industrial automation, and smart infrastructure systems. The AWR2843 is part '
 "of Texas Instruments' 77 GHz to 81 GHz mmWave sensor family, incorporating "
 'advanced features that enable superior performance in environmental sensing '
 'and high-resolution imaging.\n'
 '\n'
 'Key features of the AWR2843 include integrated transmit and receive (Tx/Rx) '
 'capabilities, facilitating robust radar functionalities with up to four '
 'transmitters and four receivers. This architecture supports a variety of '
 'radar configurations, offering spatial resolution and accurate distance '
 'measurement essential for sophisticated applications such as adaptive cruise '
 'contr

In [ ]:
# 1. Core Retrieval Strategy
# This is the main method used to find and fetch the initial set of documents.
#     "simple": A single vector search against the user's query.
#     "multi_query": Generates multiple variations of the query and combines the results (unique union).
#     "rag_fusion": Generates multiple variations of the query and combines the results using Reciprocal Rank Fusion.
#     "decomposition": Breaks a complex query into sub-questions and retrieves for each.
#     "step_back": Asks a more general question to get broader context.
#     "hyde": Generates a hypothetical document to guide the search.
#     "llm_only": No retrieval at all.

# 2. Post-Retrieval Processing
# This defines what happens to the documents after they are retrieved but before they are sent to the LLM.
#     "none": No processing; use the documents as-is.
#     "semantic_re_ranking": Use a more powerful Cross-Encoder to re-rank the documents for semantic relevance.

# 3. Final Prompting Strategy
# This determines how the LLM is instructed to use the context to formulate the final answer.
#     "strict_context": The prompt will strictly forbid the LLM from using any knowledge outside of the provided documents.
#     "permissive_context": The prompt will encourage the LLM to primarily use the context but allow it to synthesize an answer using its general knowledge if needed.

# 4. General Parameters
# These are the basic "knobs" for any given run.
#     "llm_model": e.g., "gpt-4o-mini", "gpt-4o"
#     "index_name": The specific Pinecone index to target.
#     "namespace": The namespace within that index.
#     "top_k": The number of documents to retrieve initially.
#     "debug": True or False.